In [0]:
from pyspark.sql.functions import *

df_carros = spark.read.format("csv").option("header",True).load("/aprendizado/mode_carro").distinct()

df_carros_source = df_carros.where(
    (col("id_carro")=='1')|
    (col("id_carro")=='2')|
    (col("id_carro")=='3')|
    (col("id_carro")=='7')|
    (col("id_carro")=='8')|
    (col("id_carro")=='9')
).orderBy("id_carro")

df_carros_final = df_carros.where(
    (col("id_carro")=='1')|
    (col("id_carro")=='2')|
    (col("id_carro")=='3')|
    (col("id_carro")=='4')|
    (col("id_carro")=='5')|
    (col("id_carro")=='6')
).orderBy("id_carro")

display(df_carros_source)
display(df_carros_final)

id_carro,modelo_carro,preco,cod_marca
1,Avalon,$78401.95,54
2,RDX,$95987.38,1
3,Golf,$61274.55,55
7,Voyager,$95567.75,42
8,Civic,$84749.22,20
9,Defender,$98600.79,29


id_carro,modelo_carro,preco,cod_marca
1,Avalon,$78401.95,54
2,RDX,$95987.38,1
3,Golf,$61274.55,55
4,EX,$84981.12,23
5,Escort,$77466.89,17
6,Expedition,$84698.71,17


In [0]:
df_carros_source.createOrReplaceTempView("carros_source")
df_carros_final.createOrReplaceTempView("carros_final")

In [0]:
%sql

select * 
from carros_final f
--traz o que tem em comum nas duas tabelas
where exists (select * from carros_source s where s.id_carro = f.id_carro)

--traz o que tem na final e não na source
--where not exists (select * from carros_source s where s.id_carro = f.id_carro)


id_carro,modelo_carro,preco,cod_marca
1,Avalon,$78401.95,54
2,RDX,$95987.38,1
3,Golf,$61274.55,55


In [0]:
df_exists_sql = spark.sql("""
    select * 
    from carros_final f
    where not exists (select * from carros_source s where s.id_carro = f.id_carro)
""")

display(df_exists_sql)

id_carro,modelo_carro,preco,cod_marca
4,EX,$84981.12,23
5,Escort,$77466.89,17
6,Expedition,$84698.71,17


In [0]:
#LEFTSEMI / LEFT_SEMI(PYSPARK) => EXISTS (SQL)
df_exists_spark = df_carros_final.join(df_carros_source,
                                      (df_carros_source.id_carro == df_carros_final.id_carro),
                                       "left_semi"
                                      )

#display(df_exists_spark)

#LEFTANTI / LEFT_ANTI(PYSPARK) / ANTI => NOT EXISTS (SQL)
df_exists_spark = df_carros_final.join(df_carros_source,
                                       (df_carros_source.id_carro == df_carros_final.id_carro),
                                        "ANTI"
                                       )

display(df_exists_spark)

id_carro,modelo_carro,preco,cod_marca
4,EX,$84981.12,23
5,Escort,$77466.89,17
6,Expedition,$84698.71,17
